In [57]:
import pandas as pd

In [58]:
statements = pd.read_csv('../data/processed_data/재무지표.csv')
listed = pd.read_csv('../data/processed_data/개별재무지표.csv')

---
### 전체에 대해서 값이 없는 행(기업) 탐색

In [59]:
# 모든 컬럼에서 값이 없는(결측치) 행을 찾기
empty_rows = statements[statements.iloc[:, 9:-1].isnull().all(axis=1)]

print(len(empty_rows.drop_duplicates('회사명')['회사명']))
print('------------------------------------------')
print(empty_rows.drop_duplicates('회사명')['회사명'])

35
------------------------------------------
7          (주)대호에이엘
12       (주)신화다이나믹스
74          백광산업(주)
78      삼성바이오로직스(주)
95        코스모신소재(주)
98         코스모화학(주)
105       한국전력기술(주)
114      한솔아트원제지(주)
131           (주)광무
144          (주)뉴보텍
156        (주)디지털대성
158          (주)메지온
165           (주)삼일
167           (주)상보
173         (주)서울제약
195          (주)시큐브
211           (주)씨젠
227        (주)에스에스알
248         (주)와이어블
276      (주)제이테크놀로지
288     (주)케이바이오컴퍼니
293    (주)케이에스인더스트리
298     (주)케이지모빌리언스
300      (주)케이지이니시스
308        (주)타이거일렉
311       (주)티에스넥스젠
330        (주)한진피앤씨
336         경남제약(주)
341       대한그린파워(주)
343          더라미(주)
349         디에스티(주)
352        삼영이엔씨(주)
358        씨앤티85(주)
372       우양에이치씨(주)
383       한솔아이원스(주)
Name: 회사명, dtype: object


In [60]:
print(empty_rows.shape)
print(empty_rows.drop_duplicates('회사명').shape)

(89, 194)
(35, 194)


In [61]:
# empty_rows.to_csv('재무지표null.csv',index=False, encoding='UTF-8-sig')

In [62]:
# 결측치 기업들 feature 컬럼 제외
df = empty_rows.iloc[:, :9]
df['label'] = empty_rows['label']

df.reset_index(drop=True, inplace=True)
df.head()

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,label
0,(주)대호에이엘,69460,2014,1,24,32402,1차 금속 제조업,2002-11-11,NaN,1
1,(주)신화다이나믹스,1770,2012,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
2,(주)신화다이나믹스,1770,2013,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
3,(주)신화다이나믹스,1770,2014,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
4,(주)신화다이나믹스,1770,2015,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1


### 결측치 기업 개별재무비율로 대체

In [63]:
import numpy as np

In [64]:
statements
listed

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,현금및현금성자산(*)(IFRS)(천원),...,외환차손(IFRS)(천원).1,외화환산손실(IFRS)(천원).1,지분법적용투자주식손상차손(IFRS)(천원),기부금(IFRS)(천원),투자자산처분손실(IFRS)(천원),보상비(IFRS)(천원),법인세추납액(IFRS)(천원),전기오류수정손실(IFRS)(천원),법인세비용(IFRS)(천원),label
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,5547269.0,...,2136822.0,2881977.0,0.0,0.0,0.0,0.0,0.0,0.0,1032733.0,1
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,7869589.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9539579.0,1
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,1514457.0,...,2365382.0,2147683.0,0.0,0.0,0.0,0.0,0.0,0.0,-4431729.0,1
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,NaN,323571800.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56151157.0,0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,NaN,324046180.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-127568854.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,9892897.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553201.0,1
388,한솔아이원스(주),114810,2018,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,5510953.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1544374.0,1
389,한솔아이원스(주),114810,2019,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,12307845.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2093131.0,1
390,한솔아이원스(주),114810,2020,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,9272814.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2536049.0,1


In [65]:
listed = listed.rename(columns={'공구,기구,비품(IFRS)(천원)': '공구', '조리법, 공식, 모형, 설계 및 시제품(IFRS)(천원)': '조리법', '유형,리스자산처분이익(IFRS)(천원)': '유형'})

In [66]:
def clean_column_name(col_name):
    col_name = col_name.replace('(*)', '').replace('(원)', '').replace('(백만원)', '')
    col_name = col_name.replace('[제조]', '')
    col_name = col_name.replace('(천원)', '')
    col_name = col_name.replace('(IFRS연결)', '(IFRS)')
    # col_name = col_name.replace('.1', '')

    return col_name.strip()  


listed.columns = [clean_column_name(col) for col in listed.columns]
statements.columns = [clean_column_name(col) for col in statements.columns]

In [67]:
listed['공구']

0       5699084.0
1             0.0
2       5929846.0
3      51844000.0
4      49423000.0
          ...    
387    11539743.0
388    13424068.0
389    13004596.0
390    13598336.0
391    13859285.0
Name: 공구, Length: 392, dtype: float64

In [68]:
statements.replace(0, np.nan, inplace=True)

In [69]:
df1, df2 = statements.align(listed, join='left', axis=1)

In [81]:
result = df1.fillna(df2)

In [82]:
result.columns[result.isnull().sum() > 0]

Index(['상장폐지일', '상각후원가측정 유가증권(IFRS)', '임대(주택)자산(IFRS)', '임차개량자산(IFRS)',
       '배출권자산(IFRS)', '당기법인세부채(미지급법인세)(IFRS)', '이자수익(IFRS)', '배당금수익(IFRS)',
       '사채상환이익(IFRS)', '이자비용(IFRS)', '기타대손상각비(IFRS)'],
      dtype='object')

In [83]:
listed.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)', '산업코드',
       '산업명', '상장일', '상장폐지일', '현금및현금성자산(IFRS)',
       ...
       '외환차손(IFRS).1', '외화환산손실(IFRS).1', '지분법적용투자주식손상차손(IFRS)', '기부금(IFRS)',
       '투자자산처분손실(IFRS)', '보상비(IFRS)', '법인세추납액(IFRS)', '전기오류수정손실(IFRS)',
       '법인세비용(IFRS)', 'label'],
      dtype='object', length=194)

In [84]:
statements.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)', '산업코드',
       '산업명', '상장일', '상장폐지일', '현금및현금성자산(IFRS)',
       ...
       '외환차손(IFRS).1', '외화환산손실(IFRS).1', '지분법적용투자주식손상차손(IFRS)', '기부금(IFRS)',
       '투자자산처분손실(IFRS)', '보상비(IFRS)', '법인세추납액(IFRS)', '전기오류수정손실(IFRS)',
       '법인세비용(IFRS)', 'label'],
      dtype='object', length=194)

In [86]:
cols_to_front = ['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)', '산업코드',
       '산업명', '상장일', '상장폐지일']
cols_to_end = ['label']
new_columns_order = cols_to_front + [col for col in statements.columns if col not in (cols_to_front + cols_to_end)] + cols_to_end
result = result[new_columns_order]

In [87]:
result

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,현금및현금성자산(IFRS),...,외환차손(IFRS).1,외화환산손실(IFRS).1,지분법적용투자주식손상차손(IFRS),기부금(IFRS),투자자산처분손실(IFRS),보상비(IFRS),법인세추납액(IFRS),전기오류수정손실(IFRS),법인세비용(IFRS),label
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,10376890.0,...,2136822.0,2881977.0,0.0,0.0,0.0,0.0,0.0,0.0,184992.0,1.0
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,15192647.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7248212.0,1.0
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,6564156.0,...,2365382.0,2147683.0,0.0,0.0,0.0,0.0,0.0,0.0,-696299.0,1.0
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,NaN,388440274.0,...,38000000.0,23052000.0,0.0,0.0,0.0,0.0,0.0,0.0,54918791.0,0.0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,NaN,409596263.0,...,34514000.0,23702000.0,0.0,0.0,0.0,0.0,0.0,0.0,-121688653.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,9892897.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553201.0,1.0
388,한솔아이원스(주),114810,2018,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,5510953.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1544374.0,1.0
389,한솔아이원스(주),114810,2019,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,12310161.0,...,821838.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2093131.0,1.0
390,한솔아이원스(주),114810,2020,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,9274990.0,...,799836.0,2359896.0,0.0,0.0,0.0,0.0,0.0,0.0,2536049.0,1.0


In [89]:
# result.to_csv('재무지표_개별로대체.csv',index=False,encoding='UTF-8-sig')

전체가 na인 컬럼들 제거

In [ ]:
# result_cleaned = result.dropna(axis=1, how='all')
# result_cleaned

In [90]:
cols_to_drop = result.columns[result.isna().all()]
result_cleaned = result.drop(columns=cols_to_drop)

In [91]:
cols_to_drop

Index([], dtype='object')

채워줬음에도 전체가 0인 컬럼들 제거하기

In [93]:
cols_to_drop2 = result.columns[(result==0).all()]
df_cleaned = result.drop(columns=cols_to_drop2)

In [94]:
cols_to_drop2

Index(['리스개량자산(IFRS)', '유형탐사평가자산(IFRS)', '임대주택용지(IFRS)', '미완성임대주택(IFRS)',
       '임대주택토지(IFRS)', '완성임대주택(IFRS)', '임대주택(IFRS)', '기타임대주택자산(IFRS)',
       '제호와 출판표제(IFRS)', '조리법', '*특수관계자장기매출채권(IFRS)', '장기분양미수금(IFRS)',
       '*특수관계자 장기미수금(IFRS)', '장기임대미수금(IFRS)', '장기임대주택채권(IFRS)',
       '(추정공사손실)(IFRS)', '배당건설이자(IFRS)', '파생상품평가이익(IFRS)', '파생상품평가손실(IFRS)',
       '퇴직급여적립금(IFRS)', '(대체)(IFRS)', '미분양주택관리비(IFRS)', '투자자산처분이익(IFRS)', '유형',
       '무형자산처분이익(IFRS)', '법인세환급액(IFRS)', '재고자산감모손실(IFRS)',
       '지분법적용투자주식손상차손(IFRS)', '기부금(IFRS)', '투자자산처분손실(IFRS)', '법인세추납액(IFRS)',
       '전기오류수정손실(IFRS)'],
      dtype='object')

In [95]:
# df_cleaned.to_csv('../data/processed_data/재무지표_열제거.csv', index=False, encoding='UTF-8-sig')

행이 모두 nan 값이면 지우는 코드

In [99]:

# 9번째 컬럼부터 마지막 전까지 값이 모두 NaN인 행 찾기
rows_to_remove = df_cleaned[df_cleaned.iloc[:, 9:-1].isna().all(axis=1)]

# 삭제된 행의 회사명 저장
removed_company_names = rows_to_remove.iloc[:, 0]  # 0번째 컬럼 (회사명)

# 새로운 데이터프레임에서 지운 행 제거
rows_to_keep = df_cleaned[~df_cleaned.iloc[:, 9:-1].isna().all(axis=1)]

# 결과 출력
print("삭제된 행의 회사명:")
print(removed_company_names.tolist())
print("\n새로운 데이터프레임:")
print(rows_to_keep)


삭제된 행의 회사명:
[]

새로운 데이터프레임:
            회사명   거래소코드  회계년도  소속코드  통계청 한국표준산업분류 코드 10차(대분류)   산업코드  \
0    (주)다이나믹디자인  145210  2015     1                        29  32902   
1    (주)다이나믹디자인  145210  2016     1                        29  32902   
2    (주)다이나믹디자인  145210  2017     1                        29  32902   
3       (주)대우건설   47040  2012     1                        41  64101   
4       (주)대우건설   47040  2013     1                        41  64101   
..          ...     ...   ...   ...                       ...    ...   
387   한솔아이원스(주)  114810  2017     5                        29  32902   
388   한솔아이원스(주)  114810  2018     5                        29  32902   
389   한솔아이원스(주)  114810  2019     5                        29  32902   
390   한솔아이원스(주)  114810  2020     5                        29  32902   
391   한솔아이원스(주)  114810  2021     5                        29  32902   

                   산업명         상장일 상장폐지일  현금및현금성자산(IFRS)  ...  전기오류수정이익(IFRS)  \
0      기타 기계 및 장비 제조업   20

In [98]:
# 9번째 컬럼부터 마지막 전까지 값이 모두 0인 행 찾기
rows_to_remove = df_cleaned[(df_cleaned.iloc[:, 9:-1] == 0).all(axis=1)]

# 삭제된 행의 회사명 저장
removed_company_names = rows_to_remove.iloc[:, 0]  # 0번째 컬럼 (회사명)

# 새로운 데이터프레임에서 지운 행 제거
rows_to_keep = df_cleaned[~(df_cleaned.iloc[:, 9:-1] == 0).all(axis=1)]

# 결과 출력
print("삭제된 행의 회사명:")
print(removed_company_names.tolist())
print("\n새로운 데이터프레임:")
print(rows_to_keep)

삭제된 행의 회사명:
[]

새로운 데이터프레임:
            회사명   거래소코드  회계년도  소속코드  통계청 한국표준산업분류 코드 10차(대분류)   산업코드  \
0    (주)다이나믹디자인  145210  2015     1                        29  32902   
1    (주)다이나믹디자인  145210  2016     1                        29  32902   
2    (주)다이나믹디자인  145210  2017     1                        29  32902   
3       (주)대우건설   47040  2012     1                        41  64101   
4       (주)대우건설   47040  2013     1                        41  64101   
..          ...     ...   ...   ...                       ...    ...   
387   한솔아이원스(주)  114810  2017     5                        29  32902   
388   한솔아이원스(주)  114810  2018     5                        29  32902   
389   한솔아이원스(주)  114810  2019     5                        29  32902   
390   한솔아이원스(주)  114810  2020     5                        29  32902   
391   한솔아이원스(주)  114810  2021     5                        29  32902   

                   산업명         상장일 상장폐지일  현금및현금성자산(IFRS)  ...  전기오류수정이익(IFRS)  \
0      기타 기계 및 장비 제조업   20

In [40]:
# combined_df.to_csv('../data/processed_data/재무비율_처리완.csv', index=False, encoding='UTF-8-sig')